In [3]:
import pandas as pd

In [4]:
customers_path = 'data/olist_customers_dataset.csv'
geolocation_path = 'data/olist_geolocation_dataset.csv'
order_items_path = 'data/olist_order_items_dataset.csv'
order_payments_path = 'data/olist_order_payments_dataset.csv'
order_reviews_path = 'data/olist_order_reviews_dataset.csv'
products_path = 'data/olist_products_dataset.csv'
sellers_path = 'data/olist_sellers_dataset.csv'
category_path = 'data/product_category_name_translation.csv'

In [5]:
customers_df = pd.read_csv(customers_path)
geolocation_df = pd.read_csv(geolocation_path)
order_items_df = pd.read_csv(order_items_path)
order_payments_df = pd.read_csv(order_payments_path)
order_reviews_df = pd.read_csv(order_reviews_path)
products_df = pd.read_csv(products_path)
sellers_df = pd.read_csv(sellers_path)
category_df = pd.read_csv(category_path)

tabelas = {
    'customers': customers_df,
    'geolocation': geolocation_df,
    'order_items': order_items_df,
    'order_payments': order_payments_df,
    'order_reviews': order_reviews_df,
    'products': products_df,
    'sellers': sellers_df,
    'category': category_df
}

# Visão Geral das bases


In [6]:
for nome,df in tabelas.items():
  print(f"TABELA: {nome}")
  print(df.info())


TABELA: customers
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99441 entries, 0 to 99440
Data columns (total 5 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   customer_id               99441 non-null  object
 1   customer_unique_id        99441 non-null  object
 2   customer_zip_code_prefix  99441 non-null  int64 
 3   customer_city             99441 non-null  object
 4   customer_state            99441 non-null  object
dtypes: int64(1), object(4)
memory usage: 3.8+ MB
None
TABELA: geolocation
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000163 entries, 0 to 1000162
Data columns (total 5 columns):
 #   Column                       Non-Null Count    Dtype  
---  ------                       --------------    -----  
 0   geolocation_zip_code_prefix  1000163 non-null  int64  
 1   geolocation_lat              1000163 non-null  float64
 2   geolocation_lng              1000163 non-null  float64
 3   geol

# Preparação dos Dados

In [7]:
for nome,df in tabelas.items():
  print(f"TABELA: {nome}")
  print(df.isnull().sum()) # ver quantidade de valores nulos
  print(df.duplicated().sum()) # ver quantidade de linhas duplicadas

TABELA: customers
customer_id                 0
customer_unique_id          0
customer_zip_code_prefix    0
customer_city               0
customer_state              0
dtype: int64
0
TABELA: geolocation
geolocation_zip_code_prefix    0
geolocation_lat                0
geolocation_lng                0
geolocation_city               0
geolocation_state              0
dtype: int64
261831
TABELA: order_items
order_id               0
order_item_id          0
product_id             0
seller_id              0
shipping_limit_date    0
price                  0
freight_value          0
dtype: int64
0
TABELA: order_payments
order_id                0
payment_sequential      0
payment_type            0
payment_installments    0
payment_value           0
dtype: int64
0
TABELA: order_reviews
review_id                      0
order_id                       0
review_score                   0
review_comment_title       87656
review_comment_message     58247
review_creation_date           0
review_answer_

In [8]:
# substitui valores nulos de avaliações por strings vazias '' para evitar perder o restante das colunas daquela linha principalmente o score. ajuda também a evitar problemas com tipos de dados inválidos

order_reviews_df['review_comment_title'] = order_reviews_df['review_comment_title'].fillna('')
order_reviews_df['review_comment_message'] = order_reviews_df['review_comment_message'].fillna('')

In [9]:
# como não foi informado a categoria atribuimos o valor 'desconhecido' para categorias nulas , evita perda de 610 linhas além dos campos serem menos críticos

products_df['product_category_name'] = products_df['product_category_name'].fillna('desconhecido')

products_df['product_name_lenght'] = products_df['product_name_lenght'].fillna(products_df['product_name_lenght'].median())
products_df['product_description_lenght'] = products_df['product_description_lenght'].fillna(products_df['product_description_lenght'].median())

products_df['product_photos_qty'] = products_df['product_photos_qty'].fillna(0)

# apaga 2 linhas com valores nulos além de serem valores essenciais e difíceis de imputar
'''
product_weight_g (2)
product_length_cm (2)
product_height_cm (2)
product_width_cm (2)
'''
products_df = products_df.dropna()

## Padronização de nomes de colunas


In [10]:
# corrige erro de escrita na coluna

products_df = products_df.rename(columns={
    'product_name_lenght': 'product_name_length',
    'product_description_lenght': 'product_description_length'
})

## Conversão de tipos


In [11]:
# Zip codes para string
customers_df['customer_zip_code_prefix'] = customers_df['customer_zip_code_prefix'].astype(str)
geolocation_df['geolocation_zip_code_prefix'] = geolocation_df['geolocation_zip_code_prefix'].astype(str)
sellers_df['seller_zip_code_prefix'] = sellers_df['seller_zip_code_prefix'].astype(str)

# Datas para datetime
order_items_df['shipping_limit_date'] = pd.to_datetime(order_items_df['shipping_limit_date'])
order_reviews_df['review_creation_date'] = pd.to_datetime(order_reviews_df['review_creation_date'])
order_reviews_df['review_answer_timestamp'] = pd.to_datetime(order_reviews_df['review_answer_timestamp'])


## Modelagem Relacional
